In [41]:
import numpy as np
import cv2
import glob
from sklearn.utils import shuffle
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [42]:
# Data loading and preprocessing
gen_paths = [
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset1/real/*.*',
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset2/real/*.*',
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset3/real/*.*',
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset4/real1/*.*',
    '../input/handwritten-signatures/sample_Signature/sample_Signature/genuine/*.*'
]

forg_paths = [
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset1/forge/*.*',
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset2/forge/*.*',
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset3/forge/*.*',
    '../input/handwritten-signatures/Dataset_Signature_Final/Dataset/dataset4/forge/*.*',
    '../input/handwritten-signatures/sample_Signature/sample_Signature/forged/*.*'
]


num_gen_signatures = sum([len(glob.glob(path)) for path in gen_paths])
num_forg_signatures = sum([len(glob.glob(path)) for path in forg_paths])

print("Number of genuine signatures:", num_gen_signatures)
print("Number of forged signatures:", num_forg_signatures)

Number of genuine signatures: 510
Number of forged signatures: 510


In [43]:
train_data = []
train_labels = []
test_data = []
test_labels = []

for paths in [gen_paths, forg_paths]:
    for idx, path in enumerate(paths):
        images = glob.glob(path)
        for i, image_path in enumerate(images):
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            if idx == 3:
                test_data.append(image)
                test_labels.append(0 if paths == gen_paths else 1)
            else:
                train_data.append(image)
                train_labels.append(0 if paths == gen_paths else 1)

In [44]:
train_data = np.array(train_data) / 255.0
train_labels = np.array(train_labels)
test_data = np.array(test_data) / 255.0
test_labels = np.array(test_labels)

train_data, train_labels = shuffle(train_data, train_labels)
test_data, test_labels = shuffle(test_data, test_labels)

In [45]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_datagen.fit(train_data)

In [46]:
# Base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [47]:
# Model definition
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [48]:
# Fine-tuning setup
base_model.trainable = True
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [49]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 513       
Total params: 24,637,313
Trainable params: 20,502,529
Non-trainable params: 4,134,784
_________________________________________________________________


In [50]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

In [51]:
# Define epochs
epochs = 50

# Model training
history_fine_tuned = model.fit(train_datagen.flow(train_data, train_labels, batch_size=32),
                               steps_per_epoch=len(train_data) / 32,
                               epochs=epochs,
                               validation_data=(test_data, test_labels),
                               callbacks=[early_stopping, reduce_lr])

Epoch 1/50
27/26 [==============================] - 10s 378ms/step - loss: 0.7508 - accuracy: 0.5286 - val_loss: 0.6853 - val_accuracy: 0.5556
Epoch 2/50
27/26 [==============================] - 9s 333ms/step - loss: 0.6949 - accuracy: 0.5893 - val_loss: 0.6959 - val_accuracy: 0.5500
Epoch 3/50
27/26 [==============================] - 9s 329ms/step - loss: 0.6768 - accuracy: 0.5988 - val_loss: 0.6957 - val_accuracy: 0.5333
Epoch 4/50
27/26 [==============================] - 9s 334ms/step - loss: 0.6085 - accuracy: 0.6821 - val_loss: 0.6321 - val_accuracy: 0.6778
Epoch 5/50
27/26 [==============================] - 9s 345ms/step - loss: 0.6028 - accuracy: 0.6798 - val_loss: 0.6123 - val_accuracy: 0.6611
Epoch 6/50
27/26 [==============================] - 9s 337ms/step - loss: 0.6123 - accuracy: 0.6690 - val_loss: 0.7960 - val_accuracy: 0.6500
Epoch 7/50
27/26 [==============================] - 9s 334ms/step - loss: 0.6060 - accuracy: 0.6821 - val_loss: 0.7979 - val_accuracy: 0.6278
Epoch

In [52]:
# Evaluate model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

6/6 [==============================] - 0s 73ms/step - loss: 0.4626 - accuracy: 0.8444
Test Loss: 0.4625511169433594
Test Accuracy: 0.8444444537162781


In [53]:
# Save the model
model.save('Signature_Detection_Model.h5')

In [54]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('Signature_Detection_Model.h5')

# Define function to preprocess input image
def preprocess_image(image):
    # Resize image to 224x224 (input size for ResNet model)
    image = cv2.resize(image, (224, 224))
    # Convert image to RGB (if it's in BGR)
    if image.shape[2] == 3 and image.shape[0] != 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Normalize pixel values to be between 0 and 1
    image = image / 255.0
    # Expand dimensions to match the input shape expected by the model
    image = np.expand_dims(image, axis=0)
    return image

# Define function to predict signature authenticity
def predict_signature(image):
    # Preprocess the input image
    processed_image = preprocess_image(image)
    # Make prediction
    prediction = model.predict(processed_image)
    # Convert prediction to class label (0: genuine, 1: forged)
    label = "Genuine" if prediction[0][0] < 0.5 else "Forged"
    return label, prediction[0][0]

In [57]:
# Test the model on real-time images
# Example:
image_path = '/kaggle/input/handwritten-signatures/sample_Signature/sample_Signature/genuine/NFI-00204002.png'
image = cv2.imread(image_path)
result_label, result_confidence = predict_signature(image)
print("Predicted Label:", result_label)
print("Confidence:", result_confidence)

Predicted Label: Genuine
Confidence: 0.28076544


In [56]:
# Test the model on real-time images
# Example:
image_path = '/kaggle/input/handwritten-signatures/sample_Signature/sample_Signature/forged/NFI-00101029.PNG'
image = cv2.imread(image_path)
result_label, result_confidence = predict_signature(image)
print("Predicted Label:", result_label)
print("Confidence:", result_confidence)

Predicted Label: Forged
Confidence: 0.65542483
